In [1]:
import os

import pandas as pd
import math
import random as rd
import numpy as np
import datetime as dt
import tensorflow as tf
from collections import deque
import pickle
import dlib

from deepface import DeepFace
#from deepface.basemodels import VGGFace, OpenFace, Facenet, FbDeepFace, DeepID
from deepface.extendedmodels import Age, Gender, Race, Emotion
#from deepface.modules import verification
from deepface.commons import functions, distance as dst

from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import Input,Conv2D,MaxPooling2D,Dropout,LSTM,\
                            TimeDistributed,Flatten,Dense,Bidirectional,ConvLSTM2D,MaxPooling3D,AveragePooling2D,Lambda,\
                            Activation,BatchNormalization

from tensorflow import keras
tf.test.gpu_device_name()

2024-08-08 21:03:10.170447: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


''

In [2]:
seed_constant = 27
np.random.seed(seed_constant)
rd.seed(seed_constant)
tf.random.set_seed(seed_constant)

In [3]:
DIR = '/var/scratch/mpa326/'

In [4]:
import pickle
dbfile = open('/var/scratch/mpa326/Ravdess_code/Video Features/video_train_50m_50f', 'rb')     
df = pickle.load(dbfile)

y_train_50m_50f = df['labels']
gender_train_50m_50f = df['genders']
X_train_50m_50f=df['features']
paths_train_50m_50f = df['video_files_paths']

In [5]:
dbfile = open('/var/scratch/mpa326/Ravdess_code/Video Features/video_train_40m_60f', 'rb')     
df = pickle.load(dbfile)

y_train_40m_60f = df['labels']
gender_train_40m_60f = df['genders']
X_train_40m_60f=df['features']
paths_train_40m_60f = df['video_files_paths']

In [6]:
dbfile = open('/var/scratch/mpa326/Ravdess_code/Video Features/video_train_60m_40f', 'rb')     
df = pickle.load(dbfile)

y_train_60m_40f = df['labels']
gender_train_60m_40f = df['genders']
X_train_60m_40f=df['features']
paths_train_60m_40f = df['video_files_paths']

In [7]:
dbfile = open('/var/scratch/mpa326/Ravdess_code/Video Features/video_test_common','rb')
df = pickle.load(dbfile)

y_test = df['labels']
gender_test = df['genders']
X_test=df['features']
paths_test = df['video_files_paths']

In [8]:
dbfile = open('/var/scratch/mpa326/Ravdess_code/Audio Features/audio_train_50m_50f', 'rb')     
df = pickle.load(dbfile)
X_train_aud_50m_50f=df['features']

In [9]:
dbfile = open('/var/scratch/mpa326/Ravdess_code/Audio Features/audio_train_40m_60f', 'rb')     
df = pickle.load(dbfile)
X_train_aud_40m_60f=df['features']

In [10]:
dbfile = open('/var/scratch/mpa326/Ravdess_code/Audio Features/audio_train_60m_40f', 'rb')     
df = pickle.load(dbfile)
X_train_aud_60m_40f=df['features']

In [11]:
dbfile = open('/var/scratch/mpa326/Ravdess_code/Audio Features/audio_test_common', 'rb')     
df = pickle.load(dbfile)
X_test_aud=df['features']

In [12]:
from sklearn.utils import shuffle
X_train_50m_50f, X_train_aud_50m_50f, y_train_50m_50f, gender_train_50m_50f = shuffle(X_train_50m_50f, X_train_aud_50m_50f, y_train_50m_50f, gender_train_50m_50f, random_state=27)
X_train_50m_50f, X_train_aud_50m_50f, y_train_50m_50f, gender_train_50m_50f = shuffle(X_train_50m_50f, X_train_aud_50m_50f, y_train_50m_50f, gender_train_50m_50f, random_state=7)
X_train_50m_50f, X_train_aud_50m_50f, y_train_50m_50f, gender_train_50m_50f = shuffle(X_train_50m_50f, X_train_aud_50m_50f, y_train_50m_50f, gender_train_50m_50f, random_state=42)

In [13]:
X_train_40m_60f, X_train_aud_40m_60f, y_train_40m_60f, gender_train_40m_60f = shuffle(X_train_40m_60f, X_train_aud_40m_60f, y_train_40m_60f, gender_train_40m_60f, random_state=27)
X_train_40m_60f, X_train_aud_40m_60f, y_train_40m_60f, gender_train_40m_60f = shuffle(X_train_40m_60f, X_train_aud_40m_60f, y_train_40m_60f, gender_train_40m_60f, random_state=7)
X_train_40m_60f, X_train_aud_40m_60f, y_train_40m_60f, gender_train_40m_60f = shuffle(X_train_40m_60f, X_train_aud_40m_60f, y_train_40m_60f, gender_train_40m_60f, random_state=42)

In [14]:
X_train_60m_40f, X_train_aud_60m_40f, y_train_60m_40f, gender_train_60m_40f = shuffle(X_train_60m_40f, X_train_aud_60m_40f, y_train_60m_40f, gender_train_60m_40f, random_state=27)
X_train_60m_40f, X_train_aud_60m_40f, y_train_60m_40f, gender_train_60m_40f = shuffle(X_train_60m_40f, X_train_aud_60m_40f, y_train_60m_40f, gender_train_60m_40f, random_state=7)
X_train_60m_40f, X_train_aud_60m_40f, y_train_60m_40f, gender_train_60m_40f = shuffle(X_train_60m_40f, X_train_aud_60m_40f, y_train_60m_40f, gender_train_60m_40f, random_state=42)

In [15]:
X_test, X_test_aud, y_test, gender_test = shuffle(X_test, X_test_aud, y_test, gender_test, random_state=27)
X_test, X_test_aud, y_test, gender_test = shuffle(X_test, X_test_aud, y_test, gender_test, random_state=7)
X_test, X_test_aud, y_test, gender_test = shuffle(X_test, X_test_aud, y_test, gender_test, random_state=42)

In [16]:
from tensorflow.keras.models import Model
emotion=Emotion.loadModel()
for layer in emotion.layers:
    print(layer.name)
features_emotion=Model(inputs=emotion.input, outputs=emotion.get_layer('dense_1').output)

conv2d
max_pooling2d
conv2d_1
conv2d_2
average_pooling2d
conv2d_3
conv2d_4
average_pooling2d_1
flatten
dense
dropout
dense_1
dropout_1
dense_2


In [17]:
df_ = np.ones((960,20,48,48))
for i in range(960):
  for j in range(20):
    img=X_train_50m_50f[i][j]
    img=img.astype(np.float32)
    # df_[i][j]=cv2.resize(img,(48, 48))/255
    df_[i][j]=img/255
X_train_50m_50f=df_
df_=[]

In [18]:
df_ = np.ones((960,20,48,48))
for i in range(960):
  for j in range(20):
    img=X_train_40m_60f[i][j]
    img=img.astype(np.float32)
    # df_[i][j]=cv2.resize(img,(48, 48))/255
    df_[i][j]=img/255
X_train_40m_60f=df_
df_=[]

In [19]:
df_ = np.ones((960,20,48,48))
for i in range(960):
  for j in range(20):
    img=X_train_60m_40f[i][j]
    img=img.astype(np.float32)
    # df_[i][j]=cv2.resize(img,(48, 48))/255
    df_[i][j]=img/255
X_train_60m_40f=df_
df_=[]

In [20]:
df_ = np.ones((288,20,48,48))
for i in range(288):
  for j in range(20):
    img=X_test[i][j]
    img=img.astype(np.float32)
    # df_[i][j]=cv2.resize(img,(48, 48))/255
    df_[i][j]=img/255
X_test=df_
df_=[]

In [21]:
features_emotion.trainable=False

In [22]:
class Localization(tf.keras.layers.Layer):
    def __init__(self):
        super(Localization, self).__init__()
        self.pool1 = tf.keras.layers.MaxPool2D()
        self.conv1 = tf.keras.layers.Conv2D(20, [5, 5], activation='relu')
        self.pool2 = tf.keras.layers.MaxPool2D()
        self.conv2 = tf.keras.layers.Conv2D(20, [5, 5], activation='relu')
        self.flatten = tf.keras.layers.Flatten()
        self.fc1 = tf.keras.layers.Dense(20, activation='relu')
        self.fc2 = tf.keras.layers.Dense(6, activation=None, bias_initializer=tf.keras.initializers.constant([1.0, 0.0, 0.0, 0.0, 1.0, 0.0]), kernel_initializer='zeros')

    def build(self, input_shape):
        print("Building Localization Network with input shape:", input_shape)

    def compute_output_shape(self, input_shape):
        return [None, 2, 3]

    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.pool2(x)
        x = self.flatten(x)
        x = self.fc1(x)
        theta = self.fc2(x)
        theta = tf.keras.layers.Reshape((2, 3))(theta)
        return theta

In [23]:
class BilinearInterpolation(tf.keras.layers.Layer):
    def __init__(self, height=40, width=40):
        super(BilinearInterpolation, self).__init__()
        self.height = height
        self.width = width

    def compute_output_shape(self, input_shape):
        return [None, self.height, self.width, 1]

    def get_config(self):
        return {
            'height': self.height,
            'width': self.width,
        }
    
    def build(self, input_shape):
        print("Building Bilinear Interpolation Layer with input shape:", input_shape)

    def advance_indexing(self, inputs, x, y):
        '''
        Numpy like advance indexing is not supported in tensorflow, hence, this function is a hack around the same method
        '''        
        shape = tf.shape(inputs)
        batch_size, _, _ = shape[0], shape[1], shape[2]
        
        batch_idx = tf.range(0, batch_size)
        batch_idx = tf.reshape(batch_idx, (batch_size, 1, 1))
        b = tf.tile(batch_idx, (1, self.height, self.width))
        indices = tf.stack([b, y, x], 3)
        return tf.gather_nd(inputs, indices)

    def call(self, inputs):
        images, theta = inputs
        homogenous_coordinates = self.grid_generator(batch=tf.shape(images)[0])
        return self.interpolate(images, homogenous_coordinates, theta)

    def grid_generator(self, batch):
        x = tf.linspace(-1, 1, self.width)
        y = tf.linspace(-1, 1, self.height)
            
        xx, yy = tf.meshgrid(x, y)
        xx = tf.reshape(xx, (-1,))
        yy = tf.reshape(yy, (-1,))
        homogenous_coordinates = tf.stack([xx, yy, tf.ones_like(xx)])
        homogenous_coordinates = tf.expand_dims(homogenous_coordinates, axis=0)
        homogenous_coordinates = tf.tile(homogenous_coordinates, [batch, 1, 1])
        homogenous_coordinates = tf.cast(homogenous_coordinates, dtype=tf.float32)
        return homogenous_coordinates
    
    def interpolate(self, images, homogenous_coordinates, theta):

        with tf.name_scope("Transformation"):
            transformed = tf.matmul(theta, homogenous_coordinates)
            transformed = tf.transpose(transformed, perm=[0, 2, 1])
            transformed = tf.reshape(transformed, [-1, self.height, self.width, 2])
                
            x_transformed = transformed[:, :, :, 0]
            y_transformed = transformed[:, :, :, 1]
                
            x = ((x_transformed + 1.) * tf.cast(self.width, dtype=tf.float32)) * 0.5
            y = ((y_transformed + 1.) * tf.cast(self.height, dtype=tf.float32)) * 0.5

        with tf.name_scope("VariableCasting"):
            x0 = tf.cast(tf.math.floor(x), dtype=tf.int32)
            x1 = x0 + 1
            y0 = tf.cast(tf.math.floor(y), dtype=tf.int32)
            y1 = y0 + 1

            x0 = tf.clip_by_value(x0, 0, self.width-1)
            x1 = tf.clip_by_value(x1, 0, self.width-1)
            y0 = tf.clip_by_value(y0, 0, self.height-1)
            y1 = tf.clip_by_value(y1, 0, self.height-1)
            x = tf.clip_by_value(x, 0, tf.cast(self.width, dtype=tf.float32)-1.0)
            y = tf.clip_by_value(y, 0, tf.cast(self.height, dtype=tf.float32)-1)

        with tf.name_scope("AdvanceIndexing"):
            Ia = self.advance_indexing(images, x0, y0)
            Ib = self.advance_indexing(images, x0, y1)
            Ic = self.advance_indexing(images, x1, y0)
            Id = self.advance_indexing(images, x1, y1)

        with tf.name_scope("Interpolation"):
            x0 = tf.cast(x0, dtype=tf.float32)
            x1 = tf.cast(x1, dtype=tf.float32)
            y0 = tf.cast(y0, dtype=tf.float32)
            y1 = tf.cast(y1, dtype=tf.float32)
                            
            wa = (x1-x) * (y1-y)
            wb = (x1-x) * (y-y0)
            wc = (x-x0) * (y1-y)
            wd = (x-x0) * (y-y0)

            wa = tf.expand_dims(wa, axis=3)
            wb = tf.expand_dims(wb, axis=3)
            wc = tf.expand_dims(wc, axis=3)
            wd = tf.expand_dims(wd, axis=3)
                        
        return tf.math.add_n([wa*Ia + wb*Ib + wc*Ic + wd*Id])

In [24]:
emotions_en = ["neutral", "calm", "happy", "sad", "angry", "fearful", "disgust", "surprised"]

In [25]:
def create__LRCN_with_STN() :
  image = tf.keras.layers.Input(shape=(20,48,48,1))
  theta = TimeDistributed(Localization())(image)
  x = TimeDistributed(BilinearInterpolation(height=48, width=48))([image, theta])
  features=TimeDistributed(features_emotion)(x)
  lstm=Bidirectional(LSTM(200, activation='tanh',input_shape=(20, 1024),dropout=.3))(features)
  out=Dense(len(emotions_en), activation = 'softmax')(lstm)

  return tf.keras.models.Model(inputs=image, outputs=out)


In [26]:
def get_model_name(k):
    return 'model_'+str(k)+'.h5'

In [27]:
from sklearn.model_selection import StratifiedKFold

seed = 7
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

In [28]:
#Audio

In [29]:
def model_alexnet():
  AlexNet = Sequential()

  #1st Convolutional Layer
  AlexNet.add(Conv2D(filters=96, input_shape=(128,128,3), kernel_size=(11,11), strides=(4,4), padding='same'))
  AlexNet.add(BatchNormalization())
  AlexNet.add(Activation('relu'))
  AlexNet.add(MaxPooling2D(pool_size=(2,2), strides=(1,1), padding='same'))

  #2nd Convolutional Layer
  AlexNet.add(Conv2D(filters=256, kernel_size=(5, 5), strides=(1,1), padding='same'))
  AlexNet.add(BatchNormalization())
  AlexNet.add(Activation('relu'))
  AlexNet.add(MaxPooling2D(pool_size=(2,2), strides=(1,1), padding='same'))

  #3rd Convolutional Layer
  AlexNet.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
  AlexNet.add(BatchNormalization())
  AlexNet.add(Activation('relu'))

  #4th Convolutional Layer
  AlexNet.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
  AlexNet.add(BatchNormalization())
  AlexNet.add(Activation('relu'))

  #5th Convolutional Layer
  AlexNet.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='same'))
  AlexNet.add(BatchNormalization())
  AlexNet.add(Activation('relu'))
  AlexNet.add(MaxPooling2D(pool_size=(2,2), strides=(1,1), padding='same'))

  #Passing it to a Fully Connected layer
  AlexNet.add(Flatten())
  AlexNet.add(Dense(4096, input_shape=(32,32,3,)))
  AlexNet.add(BatchNormalization())
  AlexNet.add(Activation('relu'))
  AlexNet.add(Dropout(0.4))

  # #2nd Fully Connected Layer
  # AlexNet.add(Dense(4096))
  # AlexNet.add(BatchNormalization())
  # AlexNet.add(Activation('relu'))
  # #Add Dropout
  # AlexNet.add(Dropout(0.4))

  #3rd Fully Connected Layer
  AlexNet.add(Dense(1000))
  AlexNet.add(BatchNormalization())
  AlexNet.add(Activation('relu'))
  #Add Dropout
  AlexNet.add(Dropout(0.4))

  #Output Layer
  AlexNet.add(Dense(8))
  AlexNet.add(BatchNormalization())
  AlexNet.add(Activation('softmax'))

  return AlexNet 

In [30]:
#Multimodal

In [31]:
def multi_modal_NN():
  model=Sequential()
  model.add(Input(shape=(201)))
  
  model.add(Dense(2096))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(Dropout(0.4))

  model.add(Dense(1024))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(Dropout(0.4))

  model.add(Dense(8))
  model.add(BatchNormalization())
  model.add(Activation('softmax'))
  return model

In [ ]:
#Fairness unimodal model

In [65]:
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
save_dir = os.path.abspath(DIR)+'/ravdess_50m50f_audio_model/'
final_model = model_alexnet()
final_model.compile(loss='sparse_categorical_crossentropy', optimizer='Adam', metrics=["accuracy"])
final_model.load_weights(save_dir + "model_1.h5")  # Adjust to load the best model if necessary

test_results = final_model.evaluate(X_test_aud, y_test)
test_results = dict(zip(final_model.metrics_names, test_results))

print("Test set evaluation:", test_results)

# Gender bias analysis
metrics_by_gender = {}
for gender in np.unique(gender_test):
    gender_indices = np.where(gender_test == gender)
    X_gender_test = X_test_aud[gender_indices]
    y_gender_test = y_test[gender_indices]
    y_pred = np.argmax(final_model.predict(X_gender_test), axis=1)

    accuracy = np.mean(y_pred == y_gender_test)
    precision = precision_score(y_gender_test, y_pred, average='macro')
    recall = recall_score(y_gender_test, y_pred, average='macro')
    f1 = f1_score(y_gender_test, y_pred, average='macro')

    cm = confusion_matrix(y_gender_test, y_pred)
    tn = np.sum(cm) - (np.sum(cm, axis=0) + np.sum(cm, axis=1) - np.diag(cm))
    fp = np.sum(cm, axis=0) - np.diag(cm)
    fn = np.sum(cm, axis=1) - np.diag(cm)
    tp = np.diag(cm)
    tpr = tp / (tp + fn)  # True Positive Rate
    fpr = fp / (fp + tn)  # False Positive Rate
    fnr = fn / (fn + tp)  # False Negative Rate
    precision_value = tp / (tp + fp)

    demographic_parity_dict = {}
    total_count  = len(y_pred)
    for emotion_idx in np.unique(y_pred):
        #emotion_idx = int(emotion_idx)
        count_emotion = np.count_nonzero(y_pred == emotion_idx)
        demographic_parity = count_emotion / total_count 
        emotion_name = emotions_en[emotion_idx]  # Get the emotion name from the index
        demographic_parity_dict[emotion_name] = demographic_parity

    
    metrics_by_gender[gender] = {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'tpr': tpr,
        'fpr_value': fpr,
        'fnr_value': fnr,
        'precision_value': precision_value,
        'demographic_parity': demographic_parity_dict
    }
    

#Additional fairness metrics

for gender in np.unique(gender_test):
    gender_indices = np.where(gender_test == gender)
    print(gender)
    predicted_label_1 = predicted_labels[gender_indices]
    total_count = len(predicted_label_1)
    

for gender in metrics_by_gender:
    #metrics_by_gender[gender]['demographic_parity'] = metrics_by_gender[gender]['positive_rate']
    metrics_by_gender[gender]['equalized_odds'] = (metrics_by_gender[gender]['tpr'], metrics_by_gender[gender]['fpr_value'])
    metrics_by_gender[gender]['equal_opportunity'] = metrics_by_gender[gender]['tpr']
    metrics_by_gender[gender]['predictive_parity'] = metrics_by_gender[gender]['precision_value']



9/9 [==============================] - 23s 2s/step - loss: 1.2369 - accuracy: 0.5694
Test set evaluation: {'loss': 1.2369298934936523, 'accuracy': 0.5694444179534912}
5/5 [==============================] - 9s 2s/step
female
male


In [66]:
header = ['gender', 'accuracy', 'precision', 'recall', 'f1', 'tpr', 'fpr_value', 'fnr_value', 'precision_value', 'demographic_parity']
import csv
file_path = '/var/scratch/mpa326/Ravdess_code/FairnessResults/fairness_analysis_ravdess_50m50f_audio_model.csv'

# Ensure the directory exists
os.makedirs(os.path.dirname(file_path), exist_ok=True)
# Write the dictionary to a CSV file
with open(file_path, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(header)
    
    for gender, metrics in metrics_by_gender.items():
        row = [gender]
        for key in header[1:]:  # Exclude 'gender' from header keys
            value = metrics.get(key, None)
            # Convert dictionaries to string for CSV export
            if isinstance(value, dict):
                value = str(value)
            row.append(value)
        writer.writerow(row)
    overall_row = ['overall']
    for key in header[1:]:
        # Add overall accuracy and loss, leave other columns empty (None)
        if key in test_results:
            overall_row.append(test_results[key])
        else:
            overall_row.append(None)

In [67]:
print(test_results)

{'loss': 1.2369298934936523, 'accuracy': 0.5694444179534912}


In [83]:
unique_elements, counts = np.unique(predicted_labels, return_counts=True)

print("Unique elements:", unique_elements)
print("Counts:", counts)

Unique elements: [0 1 2 3 4 5 6 7]
Counts: [29 44 37 44 28 35 42 29]


In [32]:
unique_elements, counts = np.unique(y_test, return_counts=True)

print("Unique elements:", unique_elements)
print("Counts:", counts)

Unique elements: [0 1 2 3 4 5 6 7]
Counts: [20 39 38 38 38 39 38 38]


In [ ]:
#Fairness Late Fusion

In [48]:
video_indice = 3
audio_indice = 1

In [49]:
from tensorflow.keras.models import load_model

save_dir = os.path.abspath(DIR)+'/ravdess_50m50f_audio_model/'
model_audio = model_alexnet()
model_audio.compile(loss='sparse_categorical_crossentropy', optimizer='Adam', metrics=["accuracy"])
model_audio.load_weights(save_dir +"model_"+str(audio_indice)+".h5")

2024-08-08 21:22:11.284502: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 4294967296 exceeds 10% of free system memory.


In [50]:
save_dir = os.path.abspath(DIR)+'/ravdess_50m50f_video_model/'

model_video = create__LRCN_with_STN()
model_video.compile(loss='sparese_categorical_crossentropy', optimizer='Adam', metrics=["accuracy"])
model_video.load_weights(save_dir+"model_"+str(video_indice)+".h5")

Building Localization Network with input shape: (None, 48, 48, 1)
Building Bilinear Interpolation Layer with input shape: ((None, 48, 48, 1), (None, 2, 3))


In [52]:
features_video_model=Model(inputs=model_video.input, outputs=model_video.get_layer('bidirectional_1').output)

In [54]:
features_audio_model=Model(inputs=model_audio.input, outputs=model_audio.get_layer('dropout_9').output)

In [55]:
features_video_test=features_video_model.predict(X_test)

9/9 [==============================] - 7s 327ms/step


In [56]:
features_audio_test=features_audio_model.predict(X_test_aud)

9/9 [==============================] - 19s 2s/step


In [57]:
fused_data_test = np.concatenate((features_video_test, features_audio_test), axis=1)

In [58]:
features_video=features_video_model.predict(X_train_50m_50f)

30/30 [==============================] - 10s 340ms/step


In [59]:
features_audio=features_audio_model.predict(X_train_aud_50m_50f)

30/30 [==============================] - 62s 2s/step


In [60]:
fused_data_train=np.concatenate((features_video,features_audio),axis=1)

In [61]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
fused_data_train = sc.fit_transform(fused_data_train)
fused_data_test = sc.transform(fused_data_test)

In [62]:
from sklearn.decomposition import PCA

def find_n_components(data, variance_threshold=0.95):
    """
    Determine the number of components to retain a specified amount of variance.

    Parameters:
    - data: array-like, shape (n_samples, n_features)
      The data to perform PCA on.
    - variance_threshold: float, optional (default=0.95)
      The amount of variance that needs to be retained. Should be between 0 and 1.

    Returns:
    - n_components: int
      The number of components to retain the specified amount of variance.
    """
    pca = PCA()
    pca.fit(data)
    
    # Calculate cumulative variance
    cumulative_variance = pca.explained_variance_ratio_.cumsum()
    
    # Find the number of components that satisfy the variance threshold
    n_components = (cumulative_variance >= variance_threshold).argmax() + 1
    
    return n_components

# Example usage:
# Assuming fused_data_train is your training dataset
n_components = find_n_components(fused_data_train, variance_threshold=0.95)
print(f"Number of components to retain 95% variance: {n_components}")

# Applying PCA with the determined number of components
lda = PCA(n_components=n_components)
fused_data_train = lda.fit_transform(fused_data_train)
fused_data_test = lda.transform(fused_data_test)

Number of components to retain 95% variance: 201


In [63]:
def multi_modal_NN():
  model=Sequential()
  model.add(Input(shape=(201)))
  
  model.add(Dense(2096))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(Dropout(0.4))

  model.add(Dense(1024))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(Dropout(0.4))

  model.add(Dense(8))
  model.add(BatchNormalization())
  model.add(Activation('softmax'))
  return model

In [70]:
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
save_dir = os.path.abspath(DIR)+'/ravdess_50m50f_latefusion_PCA_n201_model/'
final_model = multi_modal_NN()
final_model.compile(loss='sparse_categorical_crossentropy', optimizer='Adam', metrics=["accuracy"])
final_model.load_weights(save_dir + "model_1.h5")  # Adjust to load the best model if necessary

test_results = final_model.evaluate(fused_data_test, y_test)
test_results = dict(zip(final_model.metrics_names, test_results))

print("Test set evaluation:", test_results)

# Gender bias analysis
metrics_by_gender = {}
for gender in np.unique(gender_test):
    gender_indices = np.where(gender_test == gender)
    X_gender_test = fused_data_test[gender_indices]
    y_gender_test = y_test[gender_indices]
    y_pred = np.argmax(final_model.predict(X_gender_test), axis=1)

    accuracy = np.mean(y_pred == y_gender_test)
    precision = precision_score(y_gender_test, y_pred, average='macro')
    recall = recall_score(y_gender_test, y_pred, average='macro')
    f1 = f1_score(y_gender_test, y_pred, average='macro')

    cm = confusion_matrix(y_gender_test, y_pred)
    tn = np.sum(cm) - (np.sum(cm, axis=0) + np.sum(cm, axis=1) - np.diag(cm))
    fp = np.sum(cm, axis=0) - np.diag(cm)
    fn = np.sum(cm, axis=1) - np.diag(cm)
    tp = np.diag(cm)
    tpr = tp / (tp + fn)  # True Positive Rate
    fpr = fp / (fp + tn)  # False Positive Rate
    fnr = fn / (fn + tp)  # False Negative Rate
    precision_value = tp / (tp + fp)

    demographic_parity_dict = {}
    total_count  = len(y_pred)
    for emotion_idx in np.unique(y_pred):
        #emotion_idx = int(emotion_idx)
        count_emotion = np.count_nonzero(y_pred == emotion_idx)
        demographic_parity = count_emotion / total_count 
        emotion_name = emotions_en[emotion_idx]  # Get the emotion name from the index
        demographic_parity_dict[emotion_name] = demographic_parity

    
    metrics_by_gender[gender] = {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'tpr': tpr,
        'fpr_value': fpr,
        'fnr_value': fnr,
        'precision_value': precision_value,
        'demographic_parity': demographic_parity_dict
    }
    

#Additional fairness metrics

# for gender in np.unique(gender_test):
#     gender_indices = np.where(gender_test == gender)
#     print(gender)
#     predicted_label_1 = predicted_labels[gender_indices]
#     total_count = len(predicted_label_1)
    

for gender in metrics_by_gender:
    #metrics_by_gender[gender]['demographic_parity'] = metrics_by_gender[gender]['positive_rate']
    metrics_by_gender[gender]['equalized_odds'] = (metrics_by_gender[gender]['tpr'], metrics_by_gender[gender]['fpr_value'])
    metrics_by_gender[gender]['equal_opportunity'] = metrics_by_gender[gender]['tpr']
    metrics_by_gender[gender]['predictive_parity'] = metrics_by_gender[gender]['precision_value']



9/9 [==============================] - 0s 6ms/step - loss: 0.7417 - accuracy: 0.7986
Test set evaluation: {'loss': 0.741679310798645, 'accuracy': 0.7986111044883728}
5/5 [==============================] - 0s 5ms/step


In [71]:
header = ['gender', 'accuracy', 'precision', 'recall', 'f1', 'tpr', 'fpr_value', 'fnr_value', 'precision_value', 'demographic_parity']
import csv
file_path = '/var/scratch/mpa326/Ravdess_code/FairnessResults/fairness_analysis_ravdess_50m50f_latefusion_model.csv'

# Ensure the directory exists
os.makedirs(os.path.dirname(file_path), exist_ok=True)
# Write the dictionary to a CSV file
with open(file_path, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(header)
    
    for gender, metrics in metrics_by_gender.items():
        row = [gender]
        for key in header[1:]:  # Exclude 'gender' from header keys
            value = metrics.get(key, None)
            # Convert dictionaries to string for CSV export
            if isinstance(value, dict):
                value = str(value)
            row.append(value)
        writer.writerow(row)
    overall_row = ['overall']
    for key in header[1:]:
        # Add overall accuracy and loss, leave other columns empty (None)
        if key in test_results:
            overall_row.append(test_results[key])
        else:
            overall_row.append(None)

In [ ]:
#Fairness - Early Fusion 

In [108]:
X_train_flattened = np.reshape(X_train_40m_60f, (960, -1))

X_train_aud_flattened = np.reshape(X_train_aud_40m_60f, (960, -1))
X_train_aud_flattened.shape

(960, 49152)

In [109]:
X_test_flattened = np.reshape(X_test, (288, -1))

X_test_aud_flattened = np.reshape(X_test_aud, (288, -1))
X_test_aud_flattened.shape

(288, 49152)

In [110]:
from sklearn.preprocessing import StandardScaler
video_scaler = StandardScaler().fit(X_train_flattened)
video_data_standardized = video_scaler.fit_transform(X_train_flattened)
video_data_standardized_test = video_scaler.fit_transform(X_test_flattened)

In [111]:
audio_scaler = StandardScaler().fit(X_train_aud_flattened)
audio_data_standardized = audio_scaler.fit_transform(X_train_aud_flattened)
audio_data_standardized_test = audio_scaler.fit_transform(X_test_aud_flattened)

In [112]:
from sklearn.decomposition import PCA
pca_video = PCA(n_components=0.95).fit(video_data_standardized)
video_data_pca = pca_video.fit_transform(video_data_standardized)
video_data_pca_test = pca_video.transform(video_data_standardized_test)

In [113]:
pca_audio = PCA(n_components=0.95).fit(audio_data_standardized)
audio_data_pca = pca_audio.fit_transform(audio_data_standardized)
audio_data_pca_test = pca_audio.transform(audio_data_standardized_test)

In [118]:
audio_data_pca.shape

(960, 326)

In [119]:
video_data_pca.shape

(960, 648)

In [114]:
concatenated_test_data = np.concatenate((video_data_pca_test, audio_data_pca_test), axis=1)

In [115]:
concatenated_test_data = np.concatenate((video_data_pca_test, audio_data_pca_test), axis=1)

In [120]:
def multi_modal_NN():
  model=Sequential()
  model.add(Input(shape=(974)))
  
  model.add(Dense(2096))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(Dropout(0.4))

  model.add(Dense(1024))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(Dropout(0.4))

  model.add(Dense(8))
  model.add(BatchNormalization())
  model.add(Activation('softmax'))
  return model

In [121]:
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
save_dir = os.path.abspath(DIR)+'/ravdess_40m60f_earlyfusion_PCA_n0.95_model/'
final_model = multi_modal_NN()
final_model.compile(loss='sparse_categorical_crossentropy', optimizer='Adam', metrics=["accuracy"])
final_model.load_weights(save_dir + "model_5.h5")  # Adjust to load the best model if necessary

test_results = final_model.evaluate(concatenated_test_data, y_test)
test_results = dict(zip(final_model.metrics_names, test_results))

print("Test set evaluation:", test_results)

# Gender bias analysis
metrics_by_gender = {}
for gender in np.unique(gender_test):
    gender_indices = np.where(gender_test == gender)
    X_gender_test = concatenated_test_data[gender_indices]
    y_gender_test = y_test[gender_indices]
    y_pred = np.argmax(final_model.predict(X_gender_test), axis=1)

    accuracy = np.mean(y_pred == y_gender_test)
    precision = precision_score(y_gender_test, y_pred, average='macro')
    recall = recall_score(y_gender_test, y_pred, average='macro')
    f1 = f1_score(y_gender_test, y_pred, average='macro')

    cm = confusion_matrix(y_gender_test, y_pred)
    tn = np.sum(cm) - (np.sum(cm, axis=0) + np.sum(cm, axis=1) - np.diag(cm))
    fp = np.sum(cm, axis=0) - np.diag(cm)
    fn = np.sum(cm, axis=1) - np.diag(cm)
    tp = np.diag(cm)
    tpr = tp / (tp + fn)  # True Positive Rate
    fpr = fp / (fp + tn)  # False Positive Rate
    fnr = fn / (fn + tp)  # False Negative Rate
    precision_value = tp / (tp + fp)

    demographic_parity_dict = {}
    total_count  = len(y_pred)
    for emotion_idx in np.unique(y_pred):
        #emotion_idx = int(emotion_idx)
        count_emotion = np.count_nonzero(y_pred == emotion_idx)
        demographic_parity = count_emotion / total_count 
        emotion_name = emotions_en[emotion_idx]  # Get the emotion name from the index
        demographic_parity_dict[emotion_name] = demographic_parity

    
    metrics_by_gender[gender] = {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'tpr': tpr,
        'fpr_value': fpr,
        'fnr_value': fnr,
        'precision_value': precision_value,
        'demographic_parity': demographic_parity_dict
    }
    

#Additional fairness metrics

for gender in np.unique(gender_test):
    gender_indices = np.where(gender_test == gender)
    print(gender)
    predicted_label_1 = predicted_labels[gender_indices]
    total_count = len(predicted_label_1)
    

for gender in metrics_by_gender:
    #metrics_by_gender[gender]['demographic_parity'] = metrics_by_gender[gender]['positive_rate']
    metrics_by_gender[gender]['equalized_odds'] = (metrics_by_gender[gender]['tpr'], metrics_by_gender[gender]['fpr_value'])
    metrics_by_gender[gender]['equal_opportunity'] = metrics_by_gender[gender]['tpr']
    metrics_by_gender[gender]['predictive_parity'] = metrics_by_gender[gender]['precision_value']



9/9 [==============================] - 1s 15ms/step - loss: 1.0438 - accuracy: 0.6840
Test set evaluation: {'loss': 1.0437825918197632, 'accuracy': 0.6840277910232544}
5/5 [==============================] - 0s 14ms/step
female
male


In [122]:
header = ['gender', 'accuracy', 'precision', 'recall', 'f1', 'tpr', 'fpr_value', 'fnr_value', 'precision_value', 'demographic_parity']
import csv
file_path = '/var/scratch/mpa326/Ravdess_code/FairnessResults/fairness_analysis_ravdess_40m60f_earlyfusion_model.csv'

# Ensure the directory exists
os.makedirs(os.path.dirname(file_path), exist_ok=True)
# Write the dictionary to a CSV file
with open(file_path, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(header)
    
    for gender, metrics in metrics_by_gender.items():
        row = [gender]
        for key in header[1:]:  # Exclude 'gender' from header keys
            value = metrics.get(key, None)
            # Convert dictionaries to string for CSV export
            if isinstance(value, dict):
                value = str(value)
            row.append(value)
        writer.writerow(row)
    overall_row = ['overall']
    for key in header[1:]:
        # Add overall accuracy and loss, leave other columns empty (None)
        if key in test_results:
            overall_row.append(test_results[key])
        else:
            overall_row.append(None)

In [ ]:
#Fairness MidFusion 

In [224]:
from tensorflow.keras.models import load_model
audio_indice = 1
save_dir = os.path.abspath(DIR)+'/ravdess_40m60f_audio_model/'
model_audio = model_alexnet()
model_audio.compile(loss='sparse_categorical_crossentropy', optimizer='Adam', metrics=["accuracy"])
model_audio.load_weights(save_dir +"model_"+str(audio_indice)+".h5")

In [225]:
save_dir = os.path.abspath(DIR)+'/ravdess_40m60f_video_model/'
video_indice = 4
model_video = create__LRCN_with_STN()
model_video.compile(loss='sparese_categorical_crossentropy', optimizer='Adam', metrics=["accuracy"])
model_video.load_weights(save_dir+"model_"+str(video_indice)+".h5")

Building Localization Network with input shape: (None, 48, 48, 1)
Building Bilinear Interpolation Layer with input shape: ((None, 48, 48, 1), (None, 2, 3))


In [227]:
features_video_model=Model(inputs=model_video.input, outputs=model_video.get_layer('time_distributed_38').output)

In [228]:
features_video=features_video_model.predict(X_train_40m_60f)

30/30 [==============================] - 39s 859ms/step


In [229]:
X_train_flattened = np.reshape(features_video, (960, -1))

In [230]:
features_video_test = features_video_model.predict(X_test)

9/9 [==============================] - 4s 431ms/step


In [231]:
X_test_flattened = np.reshape(features_video_test, (288, -1))

In [232]:
features_audio_model=Model(inputs=model_audio.input, outputs=model_audio.get_layer('flatten_23').output)

In [233]:
features_audio=features_audio_model.predict(X_train_aud_40m_60f)

30/30 [==============================] - 22s 640ms/step


In [234]:
features_audio_test = features_audio_model.predict(X_test_aud)

9/9 [==============================] - 5s 600ms/step


In [235]:
from sklearn.preprocessing import StandardScaler
video_scaler = StandardScaler().fit(X_train_flattened)
video_data_standardized = video_scaler.fit_transform(X_train_flattened)
video_data_standardized_test = video_scaler.fit_transform(X_test_flattened)

In [236]:
audio_scaler = StandardScaler().fit(features_audio)
audio_data_standardized = audio_scaler.fit_transform(features_audio)
audio_data_standardized_test = audio_scaler.fit_transform(features_audio_test)

In [237]:
from sklearn.decomposition import PCA
pca_video = PCA(n_components=0.95).fit(video_data_standardized)
video_data_pca = pca_video.fit_transform(video_data_standardized)
video_data_pca_test = pca_video.transform(video_data_standardized_test)

In [238]:
pca_audio = PCA(n_components=0.95).fit(audio_data_standardized)
audio_data_pca = pca_audio.fit_transform(audio_data_standardized)
audio_data_pca_test = pca_audio.transform(audio_data_standardized_test)

In [239]:
concatenated_train_data = np.concatenate((video_data_pca, audio_data_pca), axis=1)

In [240]:
concatenated_test_data = np.concatenate((video_data_pca_test, audio_data_pca_test), axis=1)

In [241]:
def multi_modal_NN():
  model=Sequential()
  model.add(Input(shape=(1197)))
  
  model.add(Dense(2096))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(Dropout(0.4))

  model.add(Dense(1024))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(Dropout(0.4))

  model.add(Dense(8))
  model.add(BatchNormalization())
  model.add(Activation('softmax'))
  return model

In [248]:
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
save_dir = os.path.abspath(DIR)+'/ravdess_40m60f_intermediatefusion_PCA_n0.95_model/'
final_model = multi_modal_NN()
final_model.compile(loss='sparse_categorical_crossentropy', optimizer='Adam', metrics=["accuracy"])
final_model.load_weights(save_dir + "model_1.h5")  # Adjust to load the best model if necessary

test_results = final_model.evaluate(concatenated_test_data, y_test)
test_results = dict(zip(final_model.metrics_names, test_results))

print("Test set evaluation:", test_results)

# Gender bias analysis
metrics_by_gender = {}
for gender in np.unique(gender_test):
    gender_indices = np.where(gender_test == gender)
    X_gender_test = concatenated_test_data[gender_indices]
    y_gender_test = y_test[gender_indices]
    y_pred = np.argmax(final_model.predict(X_gender_test), axis=1)

    accuracy = np.mean(y_pred == y_gender_test)
    precision = precision_score(y_gender_test, y_pred, average='macro')
    recall = recall_score(y_gender_test, y_pred, average='macro')
    f1 = f1_score(y_gender_test, y_pred, average='macro')

    cm = confusion_matrix(y_gender_test, y_pred)
    tn = np.sum(cm) - (np.sum(cm, axis=0) + np.sum(cm, axis=1) - np.diag(cm))
    fp = np.sum(cm, axis=0) - np.diag(cm)
    fn = np.sum(cm, axis=1) - np.diag(cm)
    tp = np.diag(cm)
    tpr = tp / (tp + fn)  # True Positive Rate
    fpr = fp / (fp + tn)  # False Positive Rate
    fnr = fn / (fn + tp)  # False Negative Rate
    precision_value = tp / (tp + fp)

    demographic_parity_dict = {}
    total_count  = len(y_pred)
    for emotion_idx in np.unique(y_pred):
        #emotion_idx = int(emotion_idx)
        count_emotion = np.count_nonzero(y_pred == emotion_idx)
        demographic_parity = count_emotion / total_count 
        emotion_name = emotions_en[emotion_idx]  # Get the emotion name from the index
        demographic_parity_dict[emotion_name] = demographic_parity

    
    metrics_by_gender[gender] = {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'tpr': tpr,
        'fpr_value': fpr,
        'fnr_value': fnr,
        'precision_value': precision_value,
        'demographic_parity': demographic_parity_dict
    }
    

#Additional fairness metrics

for gender in np.unique(gender_test):
    gender_indices = np.where(gender_test == gender)
    print(gender)
    predicted_label_1 = predicted_labels[gender_indices]
    total_count = len(predicted_label_1)
    

for gender in metrics_by_gender:
    #metrics_by_gender[gender]['demographic_parity'] = metrics_by_gender[gender]['positive_rate']
    metrics_by_gender[gender]['equalized_odds'] = (metrics_by_gender[gender]['tpr'], metrics_by_gender[gender]['fpr_value'])
    metrics_by_gender[gender]['equal_opportunity'] = metrics_by_gender[gender]['tpr']
    metrics_by_gender[gender]['predictive_parity'] = metrics_by_gender[gender]['precision_value']



9/9 [==============================] - 1s 14ms/step - loss: 0.7430 - accuracy: 0.7847
Test set evaluation: {'loss': 0.7430149912834167, 'accuracy': 0.7847222089767456}
5/5 [==============================] - 0s 5ms/step
female
male


In [249]:
header = ['gender', 'accuracy', 'precision', 'recall', 'f1', 'tpr', 'fpr_value', 'fnr_value', 'precision_value', 'demographic_parity']
import csv
file_path = '/var/scratch/mpa326/Ravdess_code/FairnessResults/fairness_analysis_ravdess_40m60f_intermediatefusion_model.csv'

# Ensure the directory exists
os.makedirs(os.path.dirname(file_path), exist_ok=True)
# Write the dictionary to a CSV file
with open(file_path, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(header)
    
    for gender, metrics in metrics_by_gender.items():
        row = [gender]
        for key in header[1:]:  # Exclude 'gender' from header keys
            value = metrics.get(key, None)
            # Convert dictionaries to string for CSV export
            if isinstance(value, dict):
                value = str(value)
            row.append(value)
        writer.writerow(row)
    overall_row = ['overall']
    for key in header[1:]:
        # Add overall accuracy and loss, leave other columns empty (None)
        if key in test_results:
            overall_row.append(test_results[key])
        else:
            overall_row.append(None)